In [1]:
import os
import sys
this = os.getcwd()
path = this[:this.rfind("/")]
if not path in sys.path: sys.path.append(path)


import pandas as pd
import matplotlib
from datos import FEATURES_DISPONIBLES
from modelo import Modelo

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from xgboost_classifier.xgboost_predictor import XGBoost
from xgboost_regressor.xgboost_predictor import XGBoostRegressor
from promedio_zona.promedio_zona import PromedioZona
from regresion_lineal.regresion_lineal import RegresionLineal

In [3]:
class EnsambleConcatenadosPrediciendoNaNs(XGBoostRegressor):
    """
        Usa el resultado de las predicciones del modelo PromedioZona
        para entrenar y predecir con un XGBoostRegressor.
    """
    
    
    @Modelo.cronometrar()
    def __init__(self):
        self.modelo_promedios = PromedioZona()
        self.modelo_lineal = RegresionLineal()
        super().__init__()        
        
    @Modelo.cronometrar()
    def cargar_datos(self):
        self.modelo_promedios.cargar_datos()
        self.modelo_lineal.cargar_datos()
        super().cargar_datos()
    
    @Modelo.cronometrar()
    def entrenar(self):
        self.completar_nans()
        self.agregar_predicciones_modelo(self.modelo_lineal)
        self.agregar_predicciones_modelo(self.modelo_promedios)
        super().entrenar()
    
    def agregar_predicciones_modelo(self, modelo):
        columna = 'prediccion_' + modelo.modelo
        modelo.entrenar()
        predicciones_train =  modelo.predecir(modelo.train_data)
        predicciones_test = modelo.predecir(modelo.test_data)
        predicciones_submit = modelo.predecir(modelo.submit_data)
        self.train_data[columna] = predicciones_train['target']
        self.test_data[columna] = predicciones_test['target']
        self.submit_data[columna] = predicciones_submit['target']
        
    def completar_nans(self):
        """
            Completa NaNs en todos los df.
        """
        features_a_completar = ['banos', 'garages', 'habitaciones']
        dfs_a_completar = (self.train_data, self.test_data, self.submit_data)
        for feature in features_a_completar:
            modelo = XGBoost(feature)
            modelo.cargar_datos()
            modelo.entrenar()
            predicciones = modelo.predecir(modelo.submit_data)['target']
            for df in dfs_a_completar:
                df[feature].fillna(predicciones, inplace=True)

In [4]:
ensamble = EnsambleConcatenadosPrediciendoNaNs()

instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
__init__ demoro 0.0 segundos


In [5]:
ensamble.cargar_datos()

cargar demoro 1.37 segundos
cargar demoro 60.92 segundos
cargar demoro 60.97 segundos
cargar_datos demoro 127.43 segundos


In [6]:
ensamble.entrenar()

instanciar demoro 0.0 segundos
cargar demoro 61.16 segundos
entrenar demoro 0.0 segundos
entrenar demoro 2.3 segundos
predecir demoro 0.08 segundos
instanciar demoro 0.0 segundos
cargar demoro 61.2 segundos
entrenar demoro 0.0 segundos
entrenar demoro 2.16 segundos
predecir demoro 0.11 segundos
instanciar demoro 0.0 segundos
cargar demoro 60.96 segundos
entrenar demoro 0.0 segundos
entrenar demoro 4.58 segundos
predecir demoro 0.09 segundos
entrenar demoro 0.0 segundos
entrenar demoro 0.78 segundos
predecir demoro 0.13 segundos
predecir demoro 0.11 segundos
predecir demoro 0.1 segundos
entrenar demoro 0.0 segundos
entrenar demoro 0.22 segundos
predecir demoro 19.82 segundos
predecir demoro 6.75 segundos
predecir demoro 6.78 segundos
[00:56:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
entrenar demoro 0.0 segundos
entrenar demoro 287.91 segundos
entrenar demoro 516.27 segundos


In [7]:
ensamble.validar()

predecir demoro 3.08 segundos
puntuar demoro 0.0 segundos
validar demoro 3.08 segundos


622833.06

In [8]:
predicciones = ensamble.predecir(ensamble.submit_data)

predecir demoro 3.1 segundos


In [10]:
comentario = "ensamble prediciendo nans - local 622833.06"
ensamble.presentar(predicciones, comentario)

guardar demoro 0.12 segundos


100%|██████████| 967k/967k [00:07<00:00, 134kB/s]  


submit demoro 10.5 segundos
presentar demoro 10.63 segundos
